# DecisionTree Assignment - 20기 황민정

물음표 친 부분을 채우고 코드에 대한 주석을 자세하게 달아주세요!

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [15]:
def get_gini(df, label):
    total = len(df)
    counts = df[label].value_counts()
    
    gini = 1.0 # 초기값 1로 설정함
    for class_count in counts:
        gini -= (class_count / total) ** 2 # 1 - 확률값의 제곱
    
    return gini

In [16]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [25]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    list1 = list(combinations(uniques, 1))
    list2 = list(combinations(uniques, 2))
    # combinations(iterable, r) :  iterable에서 길이가 r인 모든 가능한 조합들을 반환해줌
    # 반환되는 조합은 튜플 형태
    
    result = []
    for i in (list1+list2):
        result.append(list(i)) # 원소 각각을 리스트로 만든 후 합쳐줌
    
    return result

In [26]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [35]:
def get_attribute_gini_index(df, attribute, label):
    
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)
    
    ls = []
    i = 0
    
    for a in binary_split:
        d1 = df[df[attribute].isin(a)] # a에 해당하는 데이터 프레임
        d2 = df[~df[attribute].isin(a)] # a를 뺀 데이터 프레임
        
        gini = (len(d1)/n)*(get_gini(d1, label)) + (len(d2)/n)*(get_gini(d2, label)) 
        # 해당 원소에 대한 지니계수 계산
        ls.append(gini) # 리스트에 삽입
        result[tuple(a)] = ls[i] # 딕셔너리에 삽입
        i+=1
        
    return result

In [36]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{('high',): 0.4428571428571429,
 ('medium',): 0.4583333333333333,
 ('low',): 0.45,
 ('high', 'medium'): 0.45,
 ('high', 'low'): 0.4583333333333333,
 ('medium', 'low'): 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [37]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

(('high',), 0.4428571428571429)

In [38]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

('high',)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [39]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    print("Minimum Gini Index of ", feature, ": ")
    attribute_gini_index = get_attribute_gini_index(pd_data, feature, label)
    mini_gini = min(attribute_gini_index.values())
    print(mini_gini)

Minimum Gini Index of  age : 
0.35714285714285715
Minimum Gini Index of  income : 
0.4428571428571429
Minimum Gini Index of  student : 
0.3673469387755103
Minimum Gini Index of  credit_rating : 
0.42857142857142855


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [41]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.4571428571428572,
 ('youth', 'middle_aged'): 0.4571428571428572,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [47]:
from math import log2

def getEntropy(df, feature) :
    
    data = df[feature] # df의 feature 열만 추출
    entropy = 0
    for i in data.unique():
        proportion = sum(data==i) / len(data) # sumation
        entropy -= proportion*log2(proportion)
        
    return entropy


In [48]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

In [61]:
def getGainA(df, feature) :
        
    result = {}
    columns = list(pd_data.columns[:-1]) # 목표변수 제외한 칼럼들
    
    for column in columns:
        d1 = df[[column, feature]] # 칼럼 1개와 feature 칼럼(목표변수)으로 구성된 데이터 프레임 생성
        # d1의 총 열 개수는 2개
        entropy = 0
        lst = []
        
        for a in d1.iloc[:,0].unique():
            d2 = d1[d1.iloc[:,0] == a]
            entropy += (len(d2)/len(d1))*getEntropy(d2, feature)
            e = getEntropy(df, feature) - entropy
        result[column] = e
        
    return(result)

In [62]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}